In [ ]:
import os
import torch
import pandas as pd
import numpy as np
import cv2
from scipy import ndimage
from PIL import Image
from collections import OrderedDict
from numpy.lib.function_base import angle
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
img1 = cv2.imread('/content/drive/My Drive/csc420_a3/image1.png', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('/content/drive/My Drive/csc420_a3/image2.png', cv2.IMREAD_GRAYSCALE)

In [ ]:
def get_histogram(img, occ):

  gx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
  gy = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)

  mag = cv2.magnitude(gx, gy)
  rad = np.arctan2(gy, gx)
  ang = np.rad2deg(rad)
  ang[ang < -15] += 180
  mag1 = cv2.convertScaleAbs(mag)

  threshold = 100 # tried bunch of different thresholds and trialed and errored before selecting 100. 
  #Also tried methods like Otsu. At the end 100 threshold worked perfectly for all my images based on Visualization and accuracy.
 
  mag[mag < threshold] = 0

  tau = 8
  m, n = img.shape[0] // tau, img.shape[1] // tau
  mag_histogram = np.zeros((m, n, 6), dtype=np.float32)
  occ_histogram = np.zeros((m, n, 6), dtype=np.float32)
  for i in range(m):
        for j in range(n):
            for s in range(tau):
                for t in range(tau):
                    img_i, img_j = i*tau + s, j*tau + t
                    if -15 <= ang[img_i][img_j] < 15:
                        mag_histogram[i][j][0] += mag[img_i][img_j]
                        occ_histogram[i][j][0] += 1
                    elif 15 <= ang[img_i][img_j] < 45:
                        mag_histogram[i][j][1] += mag[img_i][img_j]
                        occ_histogram[i][j][1] += 1
                    elif 45 <= ang[img_i][img_j] < 75:
                        mag_histogram[i][j][2] += mag[img_i][img_j]
                        occ_histogram[i][j][2] += 1
                    elif 75 <= ang[img_i][img_j] < 105:
                        mag_histogram[i][j][3] += mag[img_i][img_j]
                        occ_histogram[i][j][3] += 1
                    elif 105 <= ang[img_i][img_j] < 135:
                        mag_histogram[i][j][4] += mag[img_i][img_j]
                        occ_histogram[i][j][4] += 1
                    elif 135 <= ang[img_i][img_j] < 165:
                        mag_histogram[i][j][5] += mag[img_i][img_j]
                        occ_histogram[i][j][5] += 1
  if occ:
    return occ_histogram
  else:       
    return mag_histogram


In [ ]:
def plot_quiver(histogram, size=8):
    x, y, u, v = [], [], [], []
    m, n, _ = histogram.shape
    for i in range(m):
        for j in range(n):
            for l in range(6):
                x.append(j * size + size // 2)
                y.append(i * size + size // 2)
                u.append(histogram[i][j][l] * np.sin(np.pi / 6 * l))
                v.append(histogram[i][j][l] * np.cos(np.pi / 6 * l))
    return x, y, u, v

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 13))
hist_img1 = get_histogram(img1, False)
ax[0].set_title('Magnitudes')
x, y, u, v = plot_quiver(hist_img1)
ax[0].imshow(img1, cmap='gray')
ax[0].quiver(x, y, u, v, color='red', pivot='middle')
occ = get_histogram(img1, True)
ax[1].set_title('Occurrences')
x, y, u, v = plot_quiver(occ)
ax[1].imshow(img1, cmap='gray')
ax[1].quiver(x, y, u, v, color='red', pivot='middle')
plt.savefig(('/content/drive/My Drive/csc420_a3/plot5.png'))
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 13))
hist_img2 = get_histogram(img2, False)
ax[0].set_title('Magnitudes')
x, y, u, v = plot_quiver(hist_img2)
ax[0].imshow(img2, cmap='gray')
ax[0].quiver(x, y, u, v, color='red', pivot='middle')
occ = get_histogram(img2, True)
ax[1].set_title('Occurrences')
x, y, u, v = plot_quiver(occ)
ax[1].imshow(img2, cmap='gray')
ax[1].quiver(x, y, u, v, color='red', pivot='middle')
plt.savefig(('/content/drive/My Drive/csc420_a3/plot6.png'))
plt.show()

Compared to counting occurrences approach, the method of accumulating gradient magnitudes tends to produce more accurate and visually informative quiver plots around the jellyfish and bears.

Decided to go ahead with the approach of Gradient Magnitudes for the rest of the task as the reason stated above

In [ ]:
def compute_normalized_histogram(hist):
    block_size = 2
    n_orientations = 6

    e = 0.001

    n_rows, n_cols, n_bins = hist.shape

    out_rows = n_rows - block_size + 1
    out_cols = n_cols - block_size + 1

    out_hist = np.zeros((out_rows, out_cols, n_bins * block_size * block_size), dtype=np.float32)

    for i in range(out_rows):
        for j in range(out_cols):
            block = hist[i:i+block_size, j:j+block_size, :]

            block_norm = block / np.sum(np.sqrt(np.square(block) + np.square(e)))
            out_hist[i,j,:] = block_norm.reshape(-1)

    return out_hist



In [ ]:
normalized = compute_normalized_histogram(hist_img1)
np.savetxt('/content/drive/My Drive/csc420_a3/image1.txt', normalized.reshape(-1, ))
fig, ax = plt.subplots(1, 1, figsize=(15, 13))
ax.set_title('Normalized Image 1')
x, y, u, v = plot_quiver(normalized)
ax.imshow(img1, cmap='gray')
ax.quiver(x, y, u, v, color='red', pivot='middle')
plt.savefig(('/content/drive/My Drive/csc420_a3/plot7.png'))


In [ ]:
normalized_2 = compute_normalized_histogram(hist_img2)
np.savetxt('/content/drive/My Drive/csc420_a3/image2.txt', normalized_2.reshape(-1, ))
fig, ax = plt.subplots(1, 1, figsize=(15, 13))
ax.set_title('Normalized Image 2')
x, y, u, v = plot_quiver(normalized)
ax.imshow(img2, cmap='gray')
ax.quiver(x, y, u, v, color='red', pivot='middle')
plt.savefig(('/content/drive/My Drive/csc420_a3/plot8.png'))

In [ ]:
flash_img = cv2.imread('/content/drive/My Drive/csc420_a3/flash.jpg')
nonflash_img = cv2.imread('/content/drive/My Drive/csc420_a3/nonflash.jpg')
scale_factor = 0.5
flash_img = cv2.resize(flash_img, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)
nonflash_img = cv2.resize(nonflash_img, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)

flash_img = cv2.cvtColor(flash_img, cv2.COLOR_BGR2GRAY)

nonflash_img = cv2.cvtColor(nonflash_img, cv2.COLOR_BGR2GRAY)
fig, ax = plt.subplots(1, 2, figsize=(15, 13))
ax[0].set_title('Flash')
ax[0].imshow(flash_img, cmap='gray')
ax[1].set_title('Non Flash')
ax[1].imshow(nonflash_img, cmap='gray')
plt.savefig(('/content/drive/My Drive/csc420_a3/plot9.png'))

In [ ]:
flash_hist = get_histogram(flash_img, False)
nonflash_hist = get_histogram(nonflash_img, False)

fig, ax = plt.subplots(1, 2, figsize=(15, 13))

ax[0].set_title('Flash Magnitudes')
x, y, u, v = plot_quiver(flash_hist)
ax[0].imshow(flash_img, cmap='gray')
ax[0].quiver(x, y, u, v, color='red', pivot='middle')

ax[1].set_title('Non Flash Magnitudes')
x, y, u, v = plot_quiver(nonflash_hist)
ax[1].imshow(nonflash_img, cmap='gray')
ax[1].quiver(x, y, u, v, color='red', pivot='middle')
plt.savefig(('/content/drive/My Drive/csc420_a3/plot10.png'))
plt.show()

In [ ]:
normalized_flash = compute_normalized_histogram(flash_hist)
np.savetxt('/content/drive/My Drive/csc420_a3/flash.txt', normalized_flash.reshape(-1, ))
normalized_nonflash = compute_normalized_histogram(nonflash_hist)
np.savetxt('/content/drive/My Drive/csc420_a3/nonflash.txt', normalized_nonflash.reshape(-1, ))

fig, ax = plt.subplots(1, 2, figsize=(15, 13))

ax[0].set_title('Flash Normalized')
x, y, u, v = plot_quiver(normalized_flash)
ax[0].imshow(flash_img, cmap='gray')
ax[0].quiver(x, y, u, v, color='red', pivot='middle')

ax[1].set_title('Non Flash Normalized')
x, y, u, v = plot_quiver(normalized_nonflash)
ax[1].imshow(nonflash_img, cmap='gray')
ax[1].quiver(x, y, u, v, color='red', pivot='middle')
plt.savefig(('/content/drive/My Drive/csc420_a3/plot11.png'))


The gradient magnitude approach is more effective than the normalized HOG approach in detecting the edges of the rose in the flash photo. This is because the flash photo produces well-defined edges that the magnitude approach can accurately detect. On the other hand, I found that the normalized HOG approach was not beneficial in my case. This method tends to decrease the effect of large magnitudes, leading to noise in the shadow on the white wall in the non-flash photo. Additionally, the normalized HOG approach picks up on small details in the flash photo, which can be irrelevant for the analysis. I have Visually compared the outcomes of both methods on several other flash and non flash photos too and all had similar results as noted above.